In [7]:
import os
import random

# 경로 설정
base_dir = "/home/kjj73/dev_ws/yolo/yolov8/data/dataset_000/train" # images와 labels 폴더가 포함된 경로
images_dir = os.path.join(base_dir, "images")
labels_dir = os.path.join(base_dir, "labels")

# 클래스별 데이터를 확인하기 위한 딕셔너리
class_counts = {}
label_files = [f for f in os.listdir(labels_dir) if f.endswith(".txt")]

# 각 클래스의 데이터 개수 확인
for label_file in label_files:
    label_path = os.path.join(labels_dir, label_file)
    with open(label_path, "r") as f:
        lines = f.readlines()
        for line in lines:
            if line.strip():  # 빈 줄이 아닌 경우만 처리
                class_id = int(line.split()[0])  # 첫 번째 값이 클래스 ID
                class_counts[class_id] = class_counts.get(class_id, 0) + 1

# 클래스별 데이터가 없을 경우 처리
if not class_counts:
    print("Error: labels 폴더에 유효한 .txt 파일이 없거나, 파일 내용이 비어 있습니다.")
else:
    # 클래스별 데이터 개수 출력
    print("클래스별 데이터 개수:")
    for class_id, count in class_counts.items():
        print(f"클래스 {class_id}: {count}개")

    # 삭제할 비율 설정 (최소 데이터 수에 맞춤)
    min_class_count = min(class_counts.values())  # 가장 적은 클래스의 데이터 수 계산

    # 클래스별 유지할 파일 개수 계산
    class_current_counts = {class_id: 0 for class_id in class_counts}

    for label_file in label_files:
        label_path = os.path.join(labels_dir, label_file)
        with open(label_path, "r") as f:
            lines = f.readlines()

        # 수정할 데이터를 저장할 리스트
        updated_lines = []

        for line in lines:
            if line.strip():  # 빈 줄이 아닌 경우만 처리
                class_id = int(line.split()[0])
                if class_current_counts[class_id] < min_class_count:
                    updated_lines.append(line)
                    class_current_counts[class_id] += 1

        # 수정된 데이터를 파일에 다시 저장
        with open(label_path, "w") as f:
            f.writelines(updated_lines)

        # 만약 수정 후 파일이 비어 있으면 해당 파일과 이미지 삭제
        if not updated_lines:
            os.remove(label_path)
            image_file = os.path.splitext(label_file)[0] + ".jpg"
            image_path = os.path.join(images_dir, image_file)
            if os.path.exists(image_path):
                os.remove(image_path)

    print("클래스별 데이터를 가장 적은 클래스의 데이터 수에 맞추고 라벨링 없는 이미지를 제거했습니다.")


클래스별 데이터 개수:
클래스 2: 975개
클래스 0: 2330개
클래스 3: 2610개
클래스 1: 1327개
클래스별 데이터를 가장 적은 클래스의 데이터 수에 맞추고 라벨링 없는 이미지를 제거했습니다.


In [ ]:
from ultralytics import YOLO

# YOLOv8 모델 객체 생성 (기본적으로 COCO pre-trained 모델을 로드)
model = YOLO('yolov8s.pt')  # 'yolov8n.pt', 'yolov8s.pt' 등 선택 가능

# 모델 학습
model.train(
    data='/content/drive/MyDrive/Colab Notebooks/data/dataset_edit/data.yaml',
    epochs=200,
    imgsz=(320, 640),
    batch=32,
    device=0,
    mosaic=1.0,                # Mosaic 증강 활성화
    mixup=0.2,                 # MixUp 증강 비율
    degrees=15,                # 회전 각도
    translate=0.1,             # 이미지 이동 비율
    scale=0.5,                 # 크기 조정 비율
    shear=2.0,                 # 기울기 왜곡 정도
    perspective=0.0005,        # 투영 왜곡 비율
    hsv_h=0.015,               # 색조 변화 비율
    hsv_s=0.7,                 # 채도 변화 비율
    hsv_v=0.4,                 # 밝기 변화 비율
    fliplr=0.5,                # 좌우 반전 확률
    patience=15,               # 15 에포크 동안 성능 개선이 없으면 중단
    project='/content/drive/MyDrive/Colab Notebooks/'
)

# 학습 완료 후 결과 확인
results = model.val(
    data='/content/drive/MyDrive/Colab Notebooks/data/dataset_edit/data.yaml',  # 검증에 사용할 데이터셋 yaml 경로
    iou=0.7,                               # IoU threshold for validation
    save_json=True,                        # 검증 결과를 JSON으로 저장
    project='/content/drive/MyDrive/Colab Notebooks/'
)

# 학습 및 검증 결과 출력
print("Validation Results:", results)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')